In [6]:
import pandas as pd
import os
import matplotlib

In [7]:
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM


In [11]:
# Initialize LiteLLM with your OpenAI model
# llm = LiteLLM(model='ollama/qwen3:8b',api_base="http://localhost:11434", stream=False)
llm = LiteLLM(model='ollama/llama3:latest',api_base="http://localhost:11434", stream=False)

CHARTS_DIR = "exports/charts"
os.makedirs(CHARTS_DIR, exist_ok=True)

config = {
    "llm": llm,
    # "save_charts": True,
    # "save_charts_path": CHARTS_DIR,
    # "open_charts": False,
    # "enable_cache": False,
}
pai.config.set(config)

In [13]:


df_clinic_level = pd.read_csv("cc_clinic_level.csv")
df_doctor = pd.read_csv("cc_doctor.csv")
df_hourly = pd.read_csv("cc_hourly.csv")
df_patient = pd.read_csv("cc_patient.csv")

df_clinic_level = pai.DataFrame(df_clinic_level)
df_doctor = pai.DataFrame(df_doctor)
df_hourly = pai.DataFrame(df_hourly)
df_patient = pai.DataFrame(df_patient)

response=pai.chat("Which clinic have most patient?",df_clinic_level)
print(llm.model)
print(response.value)

{'type': 'string', 'value': 'The clinic with most patient is Clinic D.'}
ollama/llama3:latest
The clinic with most patient is Clinic D.


In [7]:
df_clinic_level.shape

(46, 20)

In [18]:
print(llm.model)

ollama/llama3.1:latest
